**0. Imports**

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

**1. Define PDF Function**

In [31]:
# Function to compute intensity values
def gaussian_pdf(rt, mz_array, center_rt, center_mz, sigma_rt, sigma_mz, amplitude=1):
    rt_gauss = np.exp(-0.5 * ((rt - center_rt) / sigma_rt) ** 2)
    mz_gauss = np.exp(-0.5 * ((mz_array - center_mz) / sigma_mz) ** 2)
    intensity = amplitude * rt_gauss * mz_gauss
    return intensity

In [ ]:
def create_grid_with_gaussian(rt_axis: pd.Series, mz_axis: pd.Series, peak_params: list, peak_height: float):
    # Initialize empty list to store intensity values
    intensities = []

    # Loop through each rt value and the corresponding mz values
    for rt_val, mz_array in zip(rt_axis, mz_axis):
        # mz_array is already a list or numpy array, no need to parse it again
        # If needed, ensure mz_array is a numpy array (if it's a list)
        mz_array = np.array(mz_array)

        # Initialize intensity array for this particular rt value
        rt_intensities = np.zeros(len(mz_array))
        
        # Compute intensity for each peak and accumulate it
        for peak in peak_params:
            rt_center = peak["rt_center"]
            mz_center = peak["mz_center"]
            rt_sigma = peak["rt_sigma"]
            mz_sigma = peak["mz_sigma"]
            amplitude = peak["amplitude"]

            # Apply the Gaussian function for the current peak
            peak_intensity = gaussian_pdf(rt_val, mz_array, rt_center, mz_center, rt_sigma, mz_sigma, amplitude)

            # Add the peak intensity to the total intensity
            rt_intensities += peak_intensity
        
        # Append the calculated intensities to the list
        intensities.append(rt_intensities)

    # Convert the intensities list to a DataFrame, with each rt value corresponding to a row
    intensity_df = pd.DataFrame(intensities, columns=[f"mz_{i}" for i in range(len(mz_array))])

    # Add the rt values as a new column to the DataFrame
    intensity_df["rt"] = rt_axis

    # Set the rt values as the index of the DataFrame
    intensity_df.set_index("rt", inplace=True)

    return intensity_df

**1. Define Gaussian Parameters**

In [35]:
grid = pd.read_json("grid.json")

rt_axis = grid["rt"]
mz_axis = grid["mz"]
peak_height = 25067.0
peak_params = [
    {"rt_center": 11.5, "mz_center": 152.1, "rt_sigma": 0.1, "mz_sigma": 0.03, "amplitude": 10000},
    {"rt_center": 12.0, "mz_center": 153.0, "rt_sigma": 0.15, "mz_sigma": 0.025, "amplitude": 25000},
    {"rt_center": 13.0, "mz_center": 154.5, "rt_sigma": 0.2, "mz_sigma": 0.04, "amplitude": 18000},
    # Adding more is possible
]


In [43]:
grid_with_intensity = create_grid_with_gaussian(rt_axis=rt_axis, mz_axis=mz_axis, 
                                                peak_params=peak_params, peak_height=peak_height)

print(grid_with_intensity.head())

           mz_0  mz_1  mz_2  mz_3  mz_4  mz_5  mz_6  mz_7  mz_8  mz_9  ...  \
rt                                                                     ...   
10.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
10.046676   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
10.098219   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
10.134902   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
10.166766   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

           mz_1000  mz_1001  mz_1002  mz_1003  mz_1004  mz_1005  mz_1006  \
rt                                                                         
10.000000      0.0      0.0      0.0      0.0      NaN      NaN      NaN   
10.046676      0.0      0.0      0.0      0.0      0.0      0.0      NaN   
10.098219      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
10.134902      0.0      0.0      0.0      0.0      NaN      NaN      NaN 